**Using Copernicus Marine Toolbox**
- Dataset Info: cmems_obs-oc_glo_bgc-plankton_my_l3-multi-4km_P1D
- Time Range: 4th September 1997 - 22nd September 2025
- Processing Level: Level 3
- Sensor: Harmonized Multisensor

In [1]:
import copernicusmarine
import xarray as xr
import rioxarray as rxr

In [2]:
subset_dict = {
    'dataset_id': 'cmems_obs-oc_glo_bgc-plankton_my_l3-multi-4km_P1D',
    'minimum_longitude': 38.0,
    'maximum_longitude': 47.0,
    'minimum_latitude': -6,
    'maximum_latitude': 0,
    'start_datetime': '1997/9/5',
    'end_datetime': '2025/9/22',
    'output_directory': 'C:\\Users\\ADMIN\\marine\\monsoon\\data',
    'output_filename': 'kenya_coast_ocean_color_1997_2025',
    'variables': ['CHL']
}

In [3]:
chl_data = copernicusmarine.open_dataset(dataset_id=subset_dict['dataset_id'],
                        minimum_longitude=subset_dict['minimum_longitude'],
                        maximum_longitude=subset_dict['maximum_longitude'],
                        minimum_latitude=subset_dict['minimum_latitude'],
                        maximum_latitude=subset_dict['maximum_latitude'],
                        start_datetime=subset_dict['start_datetime'],
                        end_datetime=subset_dict['end_datetime'],
                        variables=subset_dict['variables'])

INFO - 2025-09-30T12:28:11Z - Selected dataset version: "202411"
INFO - 2025-09-30T12:28:11Z - Selected dataset part: "default"


In [4]:
chl_data

<xarray.Dataset> Size: 1GB
Dimensions:    (time: 10245, latitude: 144, longitude: 216)
Coordinates:
  * latitude   (latitude) float32 576B -5.979 -5.938 -5.896 ... -0.0625 -0.02084
  * longitude  (longitude) float32 864B 38.02 38.06 38.1 ... 46.9 46.94 46.98
  * time       (time) datetime64[ns] 82kB 1997-09-05 1997-09-06 ... 2025-09-22
Data variables:
    CHL        (time, latitude, longitude) float32 1GB dask.array<chunksize=(1924, 32, 160), meta=np.ndarray>
Attributes:
    comment:      average
    source:       surface observation
    history:      Created using software developed at ACRI-ST
    institution:  ACRI
    references:   http://www.globcolour.info GlobColour has been originally f...
    Conventions:  CF-1.8, ACDD-1.3
    title:        cmems_obs-oc_glo_bgc-plankton_my_l3-multi-4km_P1D
    contact:      servicedesk.cmems@acri-st.fr

In [5]:
season_dict = {
    'north east monsoon': [1, 2, 12],
    'south east monsoon': [6, 7, 8],
    'long rains': [3, 4, 5],
    'short rains': [9, 10, 11]
}

In [6]:
def filter_by_szn(dataset:xr.Dataset, seasons_info:dict, season:str):
    filtered_ds = dataset.isel(time=dataset.time.dt.month.isin(seasons_info[season]))
    print(f"data filtered to {season} season")
    return filtered_ds

In [7]:
def composite_season(filtered_data:xr.Dataset, dimension:str):
    seasonal_composite = filtered_data.mean(dim=dimension)
    return seasonal_composite

In [8]:
def get_szn_timeseries(filtered_data:xr.Dataset, dimensions:list):
    daily_average = filtered_data.mean(dimensions)
    yearly_average = daily_average.groupby(daily_average.time.dt.year).mean()
    return yearly_average

In [ ]:
def get_monthly_averages(filtered_data:xr.Dataset, )

In [9]:
nem = filter_by_szn(dataset=chl_data, seasons_info=season_dict, season='north east monsoon')
nem

data filtered to north east monsoon season


<xarray.Dataset> Size: 314MB
Dimensions:    (time: 2527, latitude: 144, longitude: 216)
Coordinates:
  * latitude   (latitude) float32 576B -5.979 -5.938 -5.896 ... -0.0625 -0.02084
  * longitude  (longitude) float32 864B 38.02 38.06 38.1 ... 46.9 46.94 46.98
  * time       (time) datetime64[ns] 20kB 1997-12-01 1997-12-02 ... 2025-02-28
Data variables:
    CHL        (time, latitude, longitude) float32 314MB dask.array<chunksize=(1707, 32, 160), meta=np.ndarray>
Attributes:
    comment:      average
    source:       surface observation
    history:      Created using software developed at ACRI-ST
    institution:  ACRI
    references:   http://www.globcolour.info GlobColour has been originally f...
    Conventions:  CF-1.8, ACDD-1.3
    title:        cmems_obs-oc_glo_bgc-plankton_my_l3-multi-4km_P1D
    contact:      servicedesk.cmems@acri-st.fr

In [10]:
composite_season(filtered_data=nem, dimension='time')

<xarray.Dataset> Size: 126kB
Dimensions:    (latitude: 144, longitude: 216)
Coordinates:
  * latitude   (latitude) float32 576B -5.979 -5.938 -5.896 ... -0.0625 -0.02084
  * longitude  (longitude) float32 864B 38.02 38.06 38.1 ... 46.9 46.94 46.98
Data variables:
    CHL        (latitude, longitude) float32 124kB dask.array<chunksize=(32, 160), meta=np.ndarray>

In [11]:
get_szn_timeseries(filtered_data=nem, dimensions=['latitude', 'longitude'])['CHL']

<xarray.DataArray 'CHL' (year: 29)> Size: 116B
dask.array<concatenate, shape=(29,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * year     (year) int64 232B 1997 1998 1999 2000 2001 ... 2022 2023 2024 2025